# YouTube Data Collection for Weight Stigma Research

This notebook contains the data collection pipeline for analyzing YouTube videos and comments related to weight stigma research. The code implements functions to search for videos using specific keywords and collect associated comments using the YouTube Data API v3.

## Research Overview

This study investigates weight stigma in Brazilian YouTube content by analyzing:
- Video metadata for obesity-related keywords
- User comments on these videos
- Patterns in user engagement and sentiment

## Requirements

- YouTube Data API v3 key
- Python packages: pandas, google-api-python-client, tqdm, joblib
- Sufficient API quota for large-scale data collection

## Usage Instructions

1. Set your YouTube API key in the configuration section
2. Define search keywords for your research
3. Run the data collection pipeline
4. Export collected data for further analysis

**Note**: API keys should be stored securely and never committed to version control.

## 1. Import Libraries and Configuration

In [1]:
from dotenv import load_dotenv, find_dotenv

# Load environment variables from .env file
load_dotenv(find_dotenv())

True

In [2]:
import pandas as pd
import joblib
import os
import logging
from datetime import datetime
from pathlib import Path
from typing import List, Dict, Optional, Any, Tuple
from tqdm.auto import tqdm
from googleapiclient.discovery import build
import re

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s", handlers=[logging.FileHandler("youtube_data_collection.log"), logging.StreamHandler()])
logger = logging.getLogger(__name__)


# Configuration
class Config:
    """Configuration class for YouTube data collection."""

    # API Configuration
    API_KEYS = [os.getenv("YOUTUBE_API_KEY_1", ""), os.getenv("YOUTUBE_API_KEY_2", ""), os.getenv("YOUTUBE_API_KEY_3", "")]

    # Search Parameters
    SEARCH_KEYWORDS = ["obesidade", "gordo", "gorda", "obeso", "obesa"]
    LANGUAGE_CODE = "pt"
    REGION_CODE = "BR"
    PUBLISHED_AFTER = "2000-01-01T00:00:00Z"
    PUBLISHED_BEFORE = "2025-04-17T23:59:59Z"

    # File Paths
    DATA_DIR = Path("./data")
    RAW_DATA_DIR = DATA_DIR / "raw"
    TMP_DATA_DIR = DATA_DIR / "tmp"

    # API Limits
    MAX_RESULTS_PER_REQUEST = 50
    MAX_COMMENTS_PER_REQUEST = 100
    MAX_CONSECUTIVE_ERRORS = 10


# Ensure data directories exist
Config.RAW_DATA_DIR.mkdir(parents=True, exist_ok=True)
Config.TMP_DATA_DIR.mkdir(parents=True, exist_ok=True)

print("✅ Configuration loaded successfully")
print(f"📁 Data directory: {Config.DATA_DIR}")
print(f"🔍 Search keywords: {Config.SEARCH_KEYWORDS}")
print(f"🔑 API keys configured: {len([key for key in Config.API_KEYS if key])}")

# Validate API keys
if not any(Config.API_KEYS):
    print("⚠️ WARNING: No API keys found in environment variables.")
    print("Please set YOUTUBE_API_KEY_1, YOUTUBE_API_KEY_2, and/or YOUTUBE_API_KEY_3")
    print("Example: export YOUTUBE_API_KEY_1='your_api_key_here'")

✅ Configuration loaded successfully
📁 Data directory: data
🔍 Search keywords: ['obesidade', 'gordo', 'gorda', 'obeso', 'obesa']
🔑 API keys configured: 3


## 2. Core Functions for YouTube Data Collection

The following functions implement the data collection pipeline:

In [3]:
class YouTubeDataCollector:
    """
    A comprehensive YouTube data collection class for research purposes.

    This class provides methods to search for videos, collect comments,
    and retrieve video metadata using the YouTube Data API v3.
    """

    def __init__(self, api_key: str):
        """
        Initialize the YouTube data collector.

        Args:
            api_key: Valid YouTube Data API v3 key
        """
        self.api_key = api_key
        self.youtube = build("youtube", "v3", developerKey=api_key)
        logger.info(f"YouTube client initialized with API key: ...{api_key[-4:]}")

    def extract_video_id(self, url: str) -> Optional[str]:
        """
        Extract YouTube video ID from various URL formats.

        Args:
            url: YouTube video URL

        Returns:
            Video ID if found, None otherwise
        """
        patterns = [r"(?:v=|\/)([0-9A-Za-z_-]{11}).*", r"(?:embed\/)([0-9A-Za-z_-]{11})", r"(?:youtu\.be\/)([0-9A-Za-z_-]{11})"]

        for pattern in patterns:
            match = re.search(pattern, url)
            if match:
                return match.group(1)
        return None

    def search_videos(self, keyword: str, language_code: str = Config.LANGUAGE_CODE, published_after: str = Config.PUBLISHED_AFTER, published_before: str = Config.PUBLISHED_BEFORE, region_code: str = Config.REGION_CODE, max_results: Optional[int] = None) -> List[str]:
        """
        Search YouTube for videos matching a keyword.

        Args:
            keyword: Search term
            language_code: Language code for relevance
            published_after: Earliest publication date (ISO 8601)
            published_before: Latest publication date (ISO 8601)
            region_code: Country/region code
            max_results: Maximum number of results (None for all available)

        Returns:
            List of video IDs
        """
        logger.info(f"Searching videos for keyword: '{keyword}'")

        video_ids = []
        next_page_token = None
        total_requests = 0

        try:
            while True:
                search_request = self.youtube.search().list(
                    part="snippet",
                    q=keyword,
                    type="video",
                    publishedAfter=published_after,
                    publishedBefore=published_before,
                    regionCode=region_code,
                    maxResults=Config.MAX_RESULTS_PER_REQUEST,
                    pageToken=next_page_token,
                    relevanceLanguage=language_code,
                )

                response = search_request.execute()
                total_requests += 1

                # Extract video IDs
                for item in response.get("items", []):
                    video_ids.append(item["id"]["videoId"])

                # Check for pagination
                next_page_token = response.get("nextPageToken")
                if not next_page_token or (max_results and len(video_ids) >= max_results):
                    break

        except Exception as e:
            logger.error(f"Error searching videos for '{keyword}': {str(e)}")
            raise

        logger.info(f"Found {len(video_ids)} videos for '{keyword}' in {total_requests} requests")
        return video_ids[:max_results] if max_results else video_ids

    def get_video_comments(self, video_id: str) -> Optional[pd.DataFrame]:
        """
        Retrieve all comments from a YouTube video.

        Args:
            video_id: YouTube video ID

        Returns:
            DataFrame with comment data or None if error/disabled comments
        """
        logger.info(f"Collecting comments for video: {video_id}")

        comments = []
        next_page_token = None
        total_requests = 0

        try:
            while True:
                request = self.youtube.commentThreads().list(
                    part="snippet",
                    videoId=video_id,
                    textFormat="plainText",
                    maxResults=Config.MAX_COMMENTS_PER_REQUEST,
                    pageToken=next_page_token,
                )

                response = request.execute()
                total_requests += 1

                # Extract comment data
                for thread in response.get("items", []):
                    comment_data = thread["snippet"]["topLevelComment"]["snippet"]
                    comments.append(comment_data)

                # Check for pagination
                next_page_token = response.get("nextPageToken")
                if not next_page_token:
                    break

        except Exception as e:
            error_msg = str(e).lower()
            if "disabled" in error_msg or "forbidden" in error_msg:
                logger.warning(f"Comments disabled for video {video_id}")
                # Return empty DataFrame with proper structure
                return pd.DataFrame({"textDisplay": [None], "authorDisplayName": [None], "publishedAt": [None], "updatedAt": [None], "likeCount": [None], "video_id": [video_id]})
            else:
                logger.error(f"Error getting comments for {video_id}: {str(e)}")
                return None

        if comments:
            df = pd.DataFrame(comments)
            df["video_id"] = video_id
            logger.info(f"Collected {len(comments)} comments for {video_id} in {total_requests} requests")
            return df
        else:
            logger.warning(f"No comments found for video {video_id}")
            return pd.DataFrame({"video_id": [video_id]})

    def get_video_metadata(self, video_ids: List[str]) -> Dict[str, Dict[str, Any]]:
        """
        Retrieve metadata for multiple videos.

        Args:
            video_ids: List of YouTube video IDs

        Returns:
            Dictionary mapping video IDs to metadata
        """
        logger.info(f"Collecting metadata for {len(video_ids)} videos")

        metadata = {}

        # Process videos in batches of 50 (API limit)
        batch_size = 50
        for i in range(0, len(video_ids), batch_size):
            batch = video_ids[i : i + batch_size]

            try:
                request = self.youtube.videos().list(part="snippet,statistics", id=",".join(batch))
                response = request.execute()

                for item in response.get("items", []):
                    video_id = item["id"]
                    metadata[video_id] = {
                        "title": item["snippet"]["title"],
                        "description": item["snippet"]["description"],
                        "publishedAt": item["snippet"]["publishedAt"],
                        "channelTitle": item["snippet"]["channelTitle"],
                        "viewCount": item.get("statistics", {}).get("viewCount", 0),
                        "likeCount": item.get("statistics", {}).get("likeCount", 0),
                        "commentCount": item.get("statistics", {}).get("commentCount", 0),
                    }

            except Exception as e:
                logger.error(f"Error getting metadata for batch: {str(e)}")
                continue

        logger.info(f"Collected metadata for {len(metadata)} videos")
        return metadata

In [4]:
def collect_youtube_data(keywords: List[str], api_keys: List[str], output_prefix: str = None) -> Tuple[pd.DataFrame, Dict[str, Any]]:
    """
    Complete data collection pipeline for YouTube research.

    Args:
        keywords: List of search keywords
        api_keys: List of YouTube API keys (for redundancy)
        output_prefix: Prefix for output files (default: timestamp)

    Returns:
        Tuple of (comments_dataframe, collection_metadata)
    """
    if output_prefix is None:
        output_prefix = "20250417"

    # Filter valid API keys
    valid_api_keys = [key for key in api_keys if key.strip()]
    if not valid_api_keys:
        raise ValueError("No valid API keys provided")

    logger.info(f"Starting data collection with {len(keywords)} keywords and {len(valid_api_keys)} API keys")

    # Step 1: Search for videos
    all_video_ids = set()
    current_api_index = 0

    for keyword in tqdm(keywords, desc="Searching videos"):
        try:
            collector = YouTubeDataCollector(valid_api_keys[current_api_index])
            video_ids = collector.search_videos(keyword)
            all_video_ids.update(video_ids)
            logger.info(f"Keyword '{keyword}': found {len(video_ids)} videos")
        except Exception as e:
            logger.error(f"Failed to search for '{keyword}': {str(e)}")
            # Rotate to next API key
            current_api_index = (current_api_index + 1) % len(valid_api_keys)

    all_video_ids = list(all_video_ids)
    logger.info(f"Total unique videos found: {len(all_video_ids)}")

    # Save video IDs
    video_ids_file = Config.TMP_DATA_DIR / f"{output_prefix}_video_ids.joblib"
    joblib.dump(all_video_ids, video_ids_file)
    logger.info(f"Video IDs saved to: {video_ids_file}")

    # Step 2: Collect comments
    successful_videos = []
    failed_videos = []
    comment_dataframes = []
    error_count = 0

    for video_id in tqdm(all_video_ids, desc="Collecting comments"):
        try:
            collector = YouTubeDataCollector(valid_api_keys[current_api_index])
            comments_df = collector.get_video_comments(video_id)

            if comments_df is not None:
                successful_videos.append(video_id)
                comment_dataframes.append(comments_df)
                error_count = 0  # Reset error counter on success
            else:
                failed_videos.append(video_id)
                error_count += 1

        except Exception as e:
            failed_videos.append(video_id)
            logger.error(f"Failed to get comments for {video_id}: {str(e)}")
            error_count += 1

            # Rotate API key on consecutive errors
            if error_count >= Config.MAX_CONSECUTIVE_ERRORS:
                current_api_index = (current_api_index + 1) % len(valid_api_keys)
                error_count = 0
                logger.info(f"Switched to API key index: {current_api_index}")

    # Step 3: Combine and process comments
    if comment_dataframes:
        all_comments = pd.concat(comment_dataframes, ignore_index=True)

        # Clean and process data
        all_comments = all_comments.dropna(subset=["textDisplay"])
        all_comments["publishedAt"] = pd.to_datetime(all_comments["publishedAt"])
        all_comments["updatedAt"] = pd.to_datetime(all_comments["updatedAt"])
        all_comments = all_comments.sort_values(["video_id", "publishedAt"])
        all_comments = all_comments.reset_index(drop=True)

        logger.info(f"Total comments collected: {len(all_comments)}")
    else:
        all_comments = pd.DataFrame()
        logger.warning("No comments collected")

    # Step 4: Get video metadata
    metadata = {}
    if successful_videos:
        try:
            collector = YouTubeDataCollector(valid_api_keys[0])
            metadata = collector.get_video_metadata(successful_videos)

            # Add metadata to comments dataframe
            if not all_comments.empty:
                all_comments["video_title"] = all_comments["video_id"].map(lambda x: metadata.get(x, {}).get("title", "Unknown"))
        except Exception as e:
            logger.error(f"Failed to collect video metadata: {str(e)}")

    # Step 5: Save results
    if not all_comments.empty:
        # Save comments
        comments_file = Config.RAW_DATA_DIR / f"{output_prefix}_youtube_comments.parquet"
        all_comments.to_parquet(comments_file, index=False)
        logger.info(f"Comments saved to: {comments_file}")

        # Save CSV backup
        csv_file = Config.RAW_DATA_DIR / f"{output_prefix}_youtube_comments.csv"
        all_comments.to_csv(csv_file, index=False)

    # Save metadata
    metadata_file = Config.TMP_DATA_DIR / f"{output_prefix}_video_metadata.joblib"
    joblib.dump(metadata, metadata_file)

    # Save processing results
    results_file = Config.TMP_DATA_DIR / f"{output_prefix}_collection_results.joblib"
    collection_metadata = {
        "total_videos_found": len(all_video_ids),
        "successful_videos": len(successful_videos),
        "failed_videos": len(failed_videos),
        "total_comments": len(all_comments) if not all_comments.empty else 0,
        "keywords_used": keywords,
        "collection_timestamp": datetime.now().isoformat(),
        "successful_video_ids": successful_videos,
        "failed_video_ids": failed_videos,
    }
    joblib.dump(collection_metadata, results_file)

    logger.info("Data collection completed successfully!")
    logger.info(f"Results summary:")
    logger.info(f"  - Videos found: {collection_metadata['total_videos_found']}")
    logger.info(f"  - Videos with comments: {collection_metadata['successful_videos']}")
    logger.info(f"  - Failed videos: {collection_metadata['failed_videos']}")
    logger.info(f"  - Total comments: {collection_metadata['total_comments']}")

    return all_comments, collection_metadata

## 3. Data Collection Execution

Run the data collection pipeline with the configured parameters.

In [5]:
# Verify API keys before starting
if not any(Config.API_KEYS):
    print("❌ ERROR: No API keys configured!")
    print("Please set your YouTube API keys in environment variables:")
    print("export YOUTUBE_API_KEY_1='your_first_api_key'")
    print("export YOUTUBE_API_KEY_2='your_second_api_key'")
    print("export YOUTUBE_API_KEY_3='your_third_api_key'")
else:
    print("✅ API keys configured. Ready to start data collection.")
    print(f"🎯 Target keywords: {Config.SEARCH_KEYWORDS}")
    print(f"🌍 Region: {Config.REGION_CODE}, Language: {Config.LANGUAGE_CODE}")
    print(f"📅 Date range: {Config.PUBLISHED_AFTER} to {Config.PUBLISHED_BEFORE}")
    print("\n⚠️ Note: Large-scale data collection may take several hours and consume significant API quota.")
    print("Consider testing with a smaller keyword set first.")

✅ API keys configured. Ready to start data collection.
🎯 Target keywords: ['obesidade', 'gordo', 'gorda', 'obeso', 'obesa']
🌍 Region: BR, Language: pt
📅 Date range: 2000-01-01T00:00:00Z to 2025-04-17T23:59:59Z

⚠️ Note: Large-scale data collection may take several hours and consume significant API quota.
Consider testing with a smaller keyword set first.


In [ ]:
# Execute data collection
# Uncomment the following lines to run the data collection:

# comments_df, metadata = collect_youtube_data(keywords=Config.SEARCH_KEYWORDS, api_keys=Config.API_KEYS, output_prefix="weight_stigma_study")

# # Display results summary
# if not comments_df.empty:
#     print("📊 Data Collection Results:")
#     print(f"   Total comments collected: {len(comments_df):,}")
#     print(f"   Unique videos: {comments_df['video_id'].nunique():,}")
#     print(f"   Date range: {comments_df['publishedAt'].min()} to {comments_df['publishedAt'].max()}")
#     print(f"   Most active video: {comments_df['video_id'].value_counts().index[0]} ({comments_df['video_id'].value_counts().iloc[0]} comments)")

#     # Display sample data
#     print("\n📝 Sample Comments:")
#     print(comments_df[["textDisplay", "authorDisplayName", "publishedAt", "video_title"]].head())
# else:
#     print("❌ No data collected. Check API keys and network connection.")

print("💡 To run data collection, uncomment the code above and execute this cell.")
print("⏱️ Expected runtime: 1 hour depending on API quota and data volume.")

## 4. Data Exploration and Validation

After data collection, explore the collected dataset to ensure quality and completeness.

In [6]:
def explore_collected_data(comments_df: pd.DataFrame) -> None:
    """
    Provide comprehensive data exploration and quality assessment.

    Args:
        comments_df: DataFrame containing collected comments
    """
    if comments_df.empty:
        print("❌ No data to explore.")
        return

    print("🔍 DATA EXPLORATION REPORT")
    print("=" * 50)

    # Basic statistics
    print(f"📊 Dataset Overview:")
    print(f"   Total comments: {len(comments_df):,}")
    print(f"   Unique videos: {comments_df['video_id'].nunique():,}")
    print(f"   Unique authors: {comments_df['authorDisplayName'].nunique():,}")
    print(f"   Dataset shape: {comments_df.shape}")

    # Date range
    if "publishedAt" in comments_df.columns:
        date_range = comments_df["publishedAt"].agg(["min", "max"])
        print(f"\n📅 Temporal Coverage:")
        print(f"   Earliest comment: {date_range['min']}")
        print(f"   Latest comment: {date_range['max']}")
        print(f"   Time span: {(date_range['max'] - date_range['min']).days} days")

    # Comment length analysis
    if "textDisplay" in comments_df.columns:
        comments_df["comment_length"] = comments_df["textDisplay"].str.len()
        print(f"\n📝 Comment Length Statistics:")
        print(f"   Mean length: {comments_df['comment_length'].mean():.1f} characters")
        print(f"   Median length: {comments_df['comment_length'].median():.1f} characters")
        print(f"   Max length: {comments_df['comment_length'].max()} characters")

    # Top videos by comment count
    print(f"\n🎥 Most Commented Videos:")
    top_videos = comments_df["video_id"].value_counts().head()
    for i, (video_id, count) in enumerate(top_videos.items(), 1):
        title = comments_df[comments_df["video_id"] == video_id]["video_title"].iloc[0] if "video_title" in comments_df.columns else "Unknown"
        print(f"   {i}. {video_id}: {count:,} comments")
        print(f"      Title: {title[:80]}{'...' if len(str(title)) > 80 else ''}")

    # Data quality assessment
    print(f"\n🔍 Data Quality Assessment:")
    missing_text = comments_df["textDisplay"].isna().sum()
    missing_author = comments_df["authorDisplayName"].isna().sum()
    print(f"   Missing comment text: {missing_text:,} ({missing_text / len(comments_df) * 100:.1f}%)")
    print(f"   Missing author names: {missing_author:,} ({missing_author / len(comments_df) * 100:.1f}%)")

    # Engagement metrics
    if "likeCount" in comments_df.columns:
        total_likes = comments_df["likeCount"].fillna(0).sum()
        avg_likes = comments_df["likeCount"].fillna(0).mean()
        print(f"\n👍 Engagement Metrics:")
        print(f"   Total likes: {total_likes:,}")
        print(f"   Average likes per comment: {avg_likes:.2f}")
        print(f"   Most liked comment: {comments_df['likeCount'].max()} likes")


# Function to load previously collected data
def load_collected_data(file_pattern: str = "*youtube_comments.parquet") -> Optional[pd.DataFrame]:
    """
    Load previously collected YouTube data.

    Args:
        file_pattern: Glob pattern to match data files

    Returns:
        DataFrame if data found, None otherwise
    """
    from glob import glob

    data_files = list(Config.RAW_DATA_DIR.glob(file_pattern))

    if not data_files:
        print(f"❌ No data files found matching pattern: {file_pattern}")
        print(f"📁 Search directory: {Config.RAW_DATA_DIR}")
        return None

    # Load the most recent file
    latest_file = max(data_files, key=lambda x: x.stat().st_mtime)
    print(f"📂 Loading data from: {latest_file.name}")

    try:
        df = pd.read_parquet(latest_file)
        print(f"✅ Successfully loaded {len(df):,} records")
        return df
    except Exception as e:
        print(f"❌ Error loading data: {str(e)}")
        return None


# Example usage (uncomment to use):
df = load_collected_data()
if df is not None:
    explore_collected_data(df)

print("💡 Use load_collected_data() to load previously collected data")
print("💡 Use explore_collected_data(df) to analyze a dataset")

📂 Loading data from: 20250417_youtube_comments.parquet
✅ Successfully loaded 593,509 records
🔍 DATA EXPLORATION REPORT
📊 Dataset Overview:
   Total comments: 593,509
   Unique videos: 1,850
   Unique authors: 512,630
   Dataset shape: (593509, 19)

📅 Temporal Coverage:
   Earliest comment: 2006-11-24 20:16:56+00:00
   Latest comment: 2025-04-17 11:52:54+00:00
   Time span: 6718 days

📝 Comment Length Statistics:
   Mean length: 68.3 characters
   Median length: 38.0 characters
   Max length: 62137 characters

🎥 Most Commented Videos:
   1. LVb5EcOp2vw: 22,409 comments
      Title: La GORDA me llama FRIKI | #humor #risa
   2. knmVYBNj_xY: 17,325 comments
      Title: ES CRITICADA POR SER GORDA😱
   3. qohZ83lZuzg: 15,057 comments
      Title: Terrifying Night in Haunted Ghost Town | Cerro Gordo
   4. 3leulf_BVgQ: 12,679 comments
      Title: Un Día Comiendo Como La Mujer Más Gorda del Mundo (20,000 KCAL)
   5. Qwr29yZ9S-8: 12,059 comments
      Title: GORDA paródia de Tirullipa / LOKA Si

## 5. Additional Utilities and Best Practices

Helper functions for data management and research best practices.

In [7]:
def validate_api_quota(api_key: str) -> Dict[str, Any]:
    """
    Check API quota usage and limits.

    Args:
        api_key: YouTube API key to check

    Returns:
        Dictionary with quota information
    """
    try:
        youtube = build("youtube", "v3", developerKey=api_key)

        # Make a minimal request to check quota
        response = youtube.search().list(part="snippet", q="test", maxResults=1).execute()

        return {"status": "valid", "test_successful": True, "message": "API key is working correctly"}

    except Exception as e:
        error_msg = str(e).lower()
        if "quota" in error_msg:
            return {"status": "quota_exceeded", "test_successful": False, "message": "API quota exceeded. Try again later or use different key."}
        elif "invalid" in error_msg or "forbidden" in error_msg:
            return {"status": "invalid", "test_successful": False, "message": "API key is invalid or has insufficient permissions."}
        else:
            return {"status": "error", "test_successful": False, "message": f"API test failed: {str(e)}"}


# Example usage of API quota validation
if Config.API_KEYS:
    for i, key in enumerate(Config.API_KEYS):
        if key.strip():
            result = validate_api_quota(key)
            print(f"API Key {i + 1}: {result['status']}")
            print(f"  Message: {result['message']}")
            if result["test_successful"]:
                print("  ✅ API key is valid and working.")
            else:
                print("  ❌ API key validation failed.")

else:
    print("❌ No valid API keys found for quota validation.")
    print("Please set your YouTube API keys in environment variables:")
    print("export YOUTUBE_API_KEY_1='your_first_api_key'")
    print("export YOUTUBE_API_KEY_2='your_second_api_key'")
    print("export YOUTUBE_API_KEY_3='your_third_api_key'")

2025-07-21 17:22:53,316 - INFO - file_cache is only supported with oauth2client<4.0.0
2025-07-21 17:22:53,779 - WARNING - Encountered 403 Forbidden with reason "quotaExceeded"
2025-07-21 17:22:53,782 - INFO - file_cache is only supported with oauth2client<4.0.0


API Key 1: quota_exceeded
  Message: API quota exceeded. Try again later or use different key.
  ❌ API key validation failed.


2025-07-21 17:22:54,236 - WARNING - Encountered 403 Forbidden with reason "quotaExceeded"
2025-07-21 17:22:54,238 - INFO - file_cache is only supported with oauth2client<4.0.0


API Key 2: quota_exceeded
  Message: API quota exceeded. Try again later or use different key.
  ❌ API key validation failed.
API Key 3: valid
  Message: API key is working correctly
  ✅ API key is valid and working.
